# Detecting network intrusions & anomalies with higher-order topological kernels via quantum computation notebook

## 0. Imports and Dependencies

0.0. Dependencies

In [18]:
import os
from quask.core_implementation.qiskit_kernel import QiskitKernel
from quask.core import KernelType, Ansatz, KernelFactory
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler   # added
import numpy as np

0.1. Establish creds if running on IBM hardware (assumes IBM Quantum Cloud account exists and credentials saved with save_account())

In [19]:
# service = QiskitRuntimeService(instance="crn:v1:bluemix:public:quantum-computing:us-east:a/b8ff6077c08a4ea9871560ccb827d457:d3452110-b228-4c79-8959-15ea8cfd435d::") # assuming creds saved with save_account()
# backend = service.backend("ibm_rensselaer")
print("Running on Qiskit simulator (noiseless mode).")

Running on Qiskit simulator (noiseless mode).


## 1. Quantum Backend Configuration

### 1.1. Configure for either quantum simulator or hardware backend

Simulator (noiseless) backend

In [20]:
def create_qiskit_noiseless(ansatz, measurement: str, type: KernelType):
    return QiskitKernel(
        ansatz,
        measurement,
        type,
        platform="Aer",
        n_shots=None
    )

try:
    KernelFactory.add_implementation('qiskit_noiseless', create_qiskit_noiseless)
except ValueError:
    pass

KernelFactory.set_current_implementation('qiskit_noiseless')
print("KernelFactory set to Qiskit noiseless simulator.")

KernelFactory set to Qiskit noiseless simulator.


Hardware (NISQ) backend

Note: only one implementation can be selected (e.g., qiskit_noiseless OR qiskit_ibm)

## 2. Dataset Loading and Preprocessing

### 2.1. Load Dataset

In [21]:
benign_path = "500-benign.npy"
attack_path = "500-attack.npy"

qX1 = np.load(benign_path)
qX2 = np.load(attack_path)

attack_name = os.path.basename(attack_path).replace("attack", "").replace(".npy", "").upper()

print("✅ Loaded datasets:", qX1.shape, qX2.shape)

✅ Loaded datasets: (500, 8) (500, 8)


### 2.2. Feature Count

In [22]:
n_features = qX1.shape[1]
print("✅ Number of features:", n_features)

✅ Number of features: 8


### 2.3 Train/Test Split and Class Balancing

In [23]:
N = 150

qX1 = qX1[:N]
qX2 = qX2[:N]

qX = np.vstack([qX1, qX2])
qy = np.array([-1]*len(qX1) + [1]*len(qX2))

qX_train, qX_test, qy_train, qy_test = train_test_split(
    qX, qy, test_size=0.3, random_state=42, stratify=qy   # ✅ ADDED stratify
)

print("✅ Data split.")

✅ Data split.


### 2.4 Standardization

In [24]:
scaler = StandardScaler()   # ✅ CHANGED
samples = np.append(qX_train, qX_test, axis=0)
scaler.fit(samples)

qX_train = scaler.transform(qX_train)
qX_test = scaler.transform(qX_test)

print("✅ Data standardized.")

✅ Data standardized.


## 3. Quantum Kernel Construction

### 3.1 Define Quantum Ansatz

Config modified from QuASK iris dataset anomaly detection example; see [QuASK: How to optimize a quantum kernel](https://quask.readthedocs.io/en/latest/tutorials_quask/quask_2_optimizers.html) for alternate optimization techniques

In [34]:
n_features = qX1.shape[1]   

ansatz = Ansatz(
    n_features=n_features,
    n_qubits=n_features,     # important to match qubits to features
    n_operations=5
)

ansatz.initialize_to_identity()

ansatz.change_operation(0, 0, [0,1], "XX", 2.0)
ansatz.change_operation(1, 1, [2,3], "XY", 2.0)
ansatz.change_operation(2, 2, [4,5], "XZ", 2.0)
ansatz.change_operation(3, 3, [6,7], "YY", 2.0)
ansatz.change_operation(4, 4, [1,6], "YZ", 2.0)

### 3.2 Build Quantum Kernel

In [35]:
measurement = "Z" * n_features
kernel = KernelFactory.create_kernel(ansatz, measurement, KernelType.FIDELITY)
print("✅ Kernel built successfully!")

✅ Kernel built successfully!


## 4. Quantum Model – Baseline Training

### 4.1 Instantiate SVM Classifier

In [36]:
model = SVC(kernel='precomputed', C=10, class_weight='balanced')  
print("✅ SVM upgraded.")

✅ SVM upgraded.


### 4.2 Train Baseline Model

In [37]:
K_train = kernel.build_kernel(qX_train, qX_train)
model.fit(K_train, qy_train)
print("✅ Baseline model trained.")

✅ Baseline model trained.


### 4.3 Baseline Evaluation

In [38]:
K_test = kernel.build_kernel(qX_test, qX_train)
y_pred = model.predict(K_test)

baseline_acc = np.mean(qy_test == y_pred)
print("🎯 BASELINE ACCURACY:", baseline_acc)

🎯 BASELINE ACCURACY: 0.5666666666666667


## 5. Per-Attack Quantum Evaluation (All Matching .npy Pairs)

The above demo handles a singular network attack, split for improved readability. Below is our testing across all attacks.

In [39]:
from sklearn.preprocessing import StandardScaler

data_dir = '.'

for fname in os.listdir(data_dir):
    if fname.endswith('.npy') and 'benign' in fname:
        benign_path = os.path.join(data_dir, fname)
        attack_fname = fname.replace('benign', 'attack')
        attack_path = os.path.join(data_dir, attack_fname)

        attack_label = attack_fname.replace("attack", "").replace("500", "").replace(".npy", "").upper()

        if os.path.exists(attack_path):

            print(f"\n🚨 Evaluating attack: {attack_label}")

            # Load data
            qX1 = np.load(benign_path)[:50]
            qX2 = np.load(attack_path)[:50]

            qX = np.vstack([qX1, qX2])
            qy = np.array([-1]*len(qX1) + [1]*len(qX2))

            # ✅ KEEP CLASS BALANCE
            qX_train, qX_test, qy_train, qy_test = train_test_split(
                qX, qy,
                test_size=0.3,
                random_state=42,
                stratify=qy
            )

            # ✅ USE STRONG NORMALIZATION
            scaler = StandardScaler()
            samples = np.append(qX_train, qX_test, axis=0)
            scaler.fit(samples)

            qX_train = scaler.transform(qX_train)
            qX_test  = scaler.transform(qX_test)

            # ✅ TRAIN
            K_train = kernel.build_kernel(qX_train, qX_train)
            model.fit(K_train, qy_train)

            # ✅ TEST
            K_test = kernel.build_kernel(qX_test, qX_train)
            y_pred = model.predict(K_test)

            accuracy = np.mean(qy_test == y_pred)

            print(f"✅ Accuracy for {attack_label}: {accuracy:.3f}")



🚨 Evaluating attack: -
✅ Accuracy for -: 0.567


## 6. Hyperparameter Optimization – SVM C

### 6.1 Grid Search Over C

In [40]:
K_train = kernel.build_kernel(qX_train, qX_train)
K_test  = kernel.build_kernel(qX_test, qX_train)

C_values = [0.01, 0.1, 1, 10, 100, 1000]
best_C, best_acc = None, 0

print("\n🔍 Tuning SVM parameter C...\n")

for C in C_values:
    model = SVC(
        kernel="precomputed",
        C=C,
        class_weight="balanced"
    )
    model.fit(K_train, qy_train)
    preds = model.predict(K_test)
    acc = np.mean(preds == qy_test)

    print(f"C = {C:>6} → Accuracy = {acc:.4f}")

    if acc > best_acc:
        best_acc = acc
        best_C = C

print("\n✅ BEST C:", best_C)
print("🚀 TUNED ACCURACY:", best_acc)



🔍 Tuning SVM parameter C...

C =   0.01 → Accuracy = 0.4333
C =    0.1 → Accuracy = 0.4333
C =      1 → Accuracy = 0.4333
C =     10 → Accuracy = 0.5667
C =    100 → Accuracy = 0.6000
C =   1000 → Accuracy = 0.5333

✅ BEST C: 100
🚀 TUNED ACCURACY: 0.6


## 7. Cross-Validation – Model Stability

### 7.1. K-Fold Cross Validation

In [42]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []

print("\n📊 5-fold Cross-validation...\n")

for train_idx, test_idx in kf.split(qX, qy):
    X_tr_raw, X_te_raw = qX[train_idx], qX[test_idx]
    y_tr, y_te = qy[train_idx], qy[test_idx]

    # Fit scaler INSIDE fold (no leakage)
    sc = StandardScaler()
    X_tr = sc.fit_transform(X_tr_raw)
    X_te = sc.transform(X_te_raw)

    K_tr = kernel.build_kernel(X_tr, X_tr)
    K_te = kernel.build_kernel(X_te, X_tr)

    clf = SVC(
        kernel="precomputed",
        C=best_C,
        class_weight="balanced"
    )
    clf.fit(K_tr, y_tr)
    preds = clf.predict(K_te)

    scores.append(np.mean(preds == y_te))

print("✅ CV mean:", np.mean(scores))
print("✅ CV std :", np.std(scores))



📊 5-fold Cross-validation...

✅ CV mean: 0.67
✅ CV std : 0.09273618495495704
